In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cpu.html
!pip install git+https://github.com/nishant42491/PyGCL.git
!pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html


Looking in links: https://data.pyg.org/whl/torch-1.11.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.0/292.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.2/658.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.7/317.7 kB 459.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 283.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.0/467.0 kB 1.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=d751c9b6f501e9f8a0fcf132c01f2e91b15e4eff4572554d873af25b2782c7c3
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
  Cloning https://github.com/nishant42491/PyGCL.git to /tmp/pip-req-build-5zczlies
  Running command git clone --filter=blob:none --quiet https:

In [2]:
!pip install torchmetrics

In [3]:
'''import random
random.seed(10)
import numpy as np
np.random.seed(10)
import torch
torch.manual_seed(10)'''

'import random\nrandom.seed(10)\nimport numpy as np\nnp.random.seed(10)\nimport torch\ntorch.manual_seed(10)'

In [4]:
test_acc = []

In [5]:
test_acc

[]

In [6]:
a = 0

if len(test_acc) > 1:
    for i in test_acc:
        a+=i

    print(a/len(test_acc))


In [7]:
import os
import numpy as np
import torch
import torch.nn.functional as F

import torch
import torch_geometric.nn as nn
import numpy as np
import os
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import Data
import torch
import os.path as osp
import GCL.losses as L
import GCL.augmentors as A
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch.utils.data import Dataset,DataLoader
from torchmetrics import Accuracy,AUROC,F1Score

from tqdm import tqdm
from torch.optim import Adam
from GCL.eval import get_split, LREvaluator
from GCL.models import DualBranchContrast
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import pytorch_lightning as pl
import wandb
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [8]:
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
wandb_logger = WandbLogger(project="Graph_Contrastive_Learning", entity="nishant_")

wandb: Currently logged in as: nishant_. Use `wandb login --relogin` to force relogin


In [10]:
cuda = True if torch.cuda.is_available() else False

In [11]:
def prepare_trte_data(data_folder, view_list):
    num_view = len(view_list)
    labels_tr = np.loadtxt(os.path.join(data_folder, "labels_tr.csv"), delimiter=',')
    labels_te = np.loadtxt(os.path.join(data_folder, "labels_te.csv"), delimiter=',')
    labels_tr = labels_tr.astype(int)
    labels_te = labels_te.astype(int)
    data_tr_list = []
    data_te_list = []
    for i in view_list:
        data_tr_list.append(np.loadtxt(os.path.join(data_folder, str(i)+"_tr.csv"), delimiter=','))
        data_te_list.append(np.loadtxt(os.path.join(data_folder, str(i)+"_te.csv"), delimiter=','))
    num_tr = data_tr_list[0].shape[0]
    num_te = data_te_list[0].shape[0]
    data_mat_list = []
    for i in range(num_view):
        data_mat_list.append(np.concatenate((data_tr_list[i], data_te_list[i]), axis=0))
    data_tensor_list = []
    for i in range(len(data_mat_list)):
        data_tensor_list.append(torch.FloatTensor(data_mat_list[i]))
        if cuda:
            data_tensor_list[i] = data_tensor_list[i].cuda()
    idx_dict = {}
    idx_dict["tr"] = list(range(num_tr))
    idx_dict["te"] = list(range(num_tr, (num_tr+num_te)))
    data_train_list = []
    data_all_list = []
    for i in range(len(data_tensor_list)):
        data_train_list.append(data_tensor_list[i][idx_dict["tr"]].clone())
        data_all_list.append(torch.cat((data_tensor_list[i][idx_dict["tr"]].clone(),
                                       data_tensor_list[i][idx_dict["te"]].clone()),0))
    labels = np.concatenate((labels_tr, labels_te))
    
    return data_train_list, data_all_list, idx_dict, labels

In [12]:
data_folder = r'../input/graph-contrastive-learning-1/BRCA/BRCA'
view_list = [1,2,3]

In [21]:
data_tr_list, data_trte_list, trte_idx, labels_trte = prepare_trte_data(data_folder, view_list)

In [14]:
cuda = True if torch.cuda.is_available() else False

def cal_sample_weight(labels, num_class, use_sample_weight=True):
    if not use_sample_weight:
        return np.ones(len(labels)) / len(labels)
    count = np.zeros(num_class)
    for i in range(num_class):
        count[i] = np.sum(labels==i)
    sample_weight = np.zeros(labels.shape)
    for i in range(num_class):
        sample_weight[np.where(labels==i)[0]] = count[i]/np.sum(count)
    
    return sample_weight


def one_hot_tensor(y, num_dim):
    y_onehot = torch.zeros(y.shape[0], num_dim)
    y_onehot.scatter_(1, y.view(-1,1), 1)
    
    return y_onehot


def cosine_distance_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    return 1 - torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)


def to_sparse(x):
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)
    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())


def cal_adj_mat_parameter(edge_per_node, data, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    parameter = torch.sort(dist.reshape(-1,)).values[edge_per_node*data.shape[0]]
    return np.ndarray.item(parameter.data.cpu().numpy())


def graph_from_dist_tensor(dist, parameter, self_dist=True):
    if self_dist:
        assert dist.shape[0]==dist.shape[1], "Input is not pairwise dist matrix"
    g = (dist <= parameter).float()
    if self_dist:
        diag_idx = np.diag_indices(g.shape[0])
        g[diag_idx[0], diag_idx[1]] = 0
        
    return g


def gen_adj_mat_tensor(data, parameter, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    g = graph_from_dist_tensor(dist, parameter, self_dist=True)
    if metric == "cosine":
        adj = 1-dist
    else:
        raise NotImplementedError
    adj = adj*g 
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    if cuda:
        I = I.cuda()
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    adj = to_sparse(adj)
    
    return adj


def gen_test_adj_mat_tensor(data, trte_idx, parameter, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    adj = torch.zeros((data.shape[0], data.shape[0]))
    if cuda:
        adj = adj.cuda()
    num_tr = len(trte_idx["tr"])
    
    dist_tr2te = cosine_distance_torch(data[trte_idx["tr"]], data[trte_idx["te"]])
    g_tr2te = graph_from_dist_tensor(dist_tr2te, parameter, self_dist=False)
    if metric == "cosine":
        adj[:num_tr,num_tr:] = 1-dist_tr2te
    else:
        raise NotImplementedError
    adj[:num_tr,num_tr:] = adj[:num_tr,num_tr:]*g_tr2te
    
    dist_te2tr = cosine_distance_torch(data[trte_idx["te"]], data[trte_idx["tr"]])
    g_te2tr = graph_from_dist_tensor(dist_te2tr, parameter, self_dist=False)
    if metric == "cosine":
        adj[num_tr:,:num_tr] = 1-dist_te2tr
    else:
        raise NotImplementedError
    adj[num_tr:,:num_tr] = adj[num_tr:,:num_tr]*g_te2tr # retain selected edges
    
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    if cuda:
        I = I.cuda()
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    adj = to_sparse(adj)
    
    return adj


def save_model_dict(folder, model_dict):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for module in model_dict:
        torch.save(model_dict[module].state_dict(), os.path.join(folder, module+".pth"))
            
    
def load_model_dict(folder, model_dict):
    for module in model_dict:
        if os.path.exists(os.path.join(folder, module+".pth")):
#            print("Module {:} loaded!".format(module))
            model_dict[module].load_state_dict(torch.load(os.path.join(folder, module+".pth"), map_location="cuda:{:}".format(torch.cuda.current_device())))
        else:
            print("WARNING: Module {:} from model_dict is not loaded!".format(module))
        if cuda:
            model_dict[module].cuda()    
    return model_dict



def gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter):
    adj_metric = "cosine" # cosine distance
    adj_train_list = []
    adj_test_list = []
    for i in range(len(data_tr_list)):
        adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, data_tr_list[i], adj_metric)
        adj_train_list.append(gen_adj_mat_tensor(data_tr_list[i], adj_parameter_adaptive, adj_metric))
        adj_test_list.append(gen_test_adj_mat_tensor(data_trte_list[i], trte_idx, adj_parameter_adaptive, adj_metric))
    
    return adj_train_list, adj_test_list

In [32]:
adj_parameter = 2
adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)

In [33]:
data_tr_list

[tensor([[0.2823, 0.1148, 0.5080,  ..., 0.3504, 0.4459, 0.2945],
         [0.2963, 0.0542, 0.5594,  ..., 0.2358, 0.6009, 0.3951],
         [0.2352, 0.3170, 0.5059,  ..., 0.1575, 0.4421, 0.4063],
         ...,
         [0.1065, 0.0217, 0.3142,  ..., 0.3878, 0.4653, 0.3595],
         [0.2444, 0.1371, 0.5480,  ..., 0.1654, 0.6213, 0.3647],
         [0.4102, 0.1096, 0.5818,  ..., 0.2716, 0.5736, 0.3803]]),
 tensor([[0.7615, 0.8605, 0.6642,  ..., 0.5645, 0.5360, 0.5845],
         [0.8281, 0.8584, 0.6465,  ..., 0.6303, 0.5936, 0.2596],
         [0.8959, 0.8155, 0.5906,  ..., 0.8908, 0.6695, 0.1482],
         ...,
         [0.6335, 0.5735, 0.3636,  ..., 0.4891, 0.2417, 0.1640],
         [0.8527, 0.8258, 0.5873,  ..., 0.7349, 0.4855, 0.4430],
         [0.8407, 0.8062, 0.6367,  ..., 0.5428, 0.3078, 0.2428]]),
 tensor([[0.7256, 0.7770, 0.7261,  ..., 0.3342, 0.4480, 0.8130],
         [0.7236, 0.7750, 0.7238,  ..., 0.3470, 0.4269, 0.7430],
         [0.7073, 0.7592, 0.7078,  ..., 0.2833, 0.5421, 0.

In [38]:
adj_tr_list[0]

tensor(indices=tensor([[  0,   1,   2,  ..., 872, 873, 874],
                       [  0,   1,   2,  ..., 872, 873, 874]]),
       values=tensor([1.0000, 1.0000, 0.2505,  ..., 1.0000, 1.0000, 1.0000]),
       size=(875, 875), nnz=1755, layout=torch.sparse_coo)

In [35]:
train_data_list = []
test_data_list = []

In [ ]:
for i in range(len(adj_tr_list)):
    y = torch.tensor(labels_trte)
    cur_data = Data(x=data_trte_list[i],
                    edge_index=adj_tr_list[i].coalesce().indices(),
                    edge_weight=adj_tr_list[i].coalesce().values(),
                    y = y)
    train_data_list.append(cur_data)
    
    '''cur_test_data = Data(x=data_trte_list[i][np.array(trte_idx['te'])],
                    edge_index=adj_te_list[i].coalesce().indices(),
                    edge_weight=adj_te_list[i].coalesce().values(),
                    y = labels_trte[np.array(trte_idx['te'])])
    test_data_list.append(cur_test_data)'''

In [ ]:
num_epochs = 2000
class GConv(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, activation, num_layers):
        super(GConv, self).__init__()
        self.activation = activation()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(input_dim, hidden_dim, cached=False))
        for _ in range(num_layers - 1):
            self.layers.append(GCNConv(hidden_dim, hidden_dim, cached=False))

    def forward(self, x, edge_index, edge_weight=None):
        z = x
        for i, conv in enumerate(self.layers):
            z = conv(z, edge_index, edge_weight)
            z = self.activation(z)
        return z


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor, hidden_dim, proj_dim):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

        self.fc1 = torch.nn.Linear(hidden_dim, proj_dim)
        self.fc2 = torch.nn.Linear(proj_dim, hidden_dim)

    def forward(self, x, edge_index, edge_weight=None):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index, edge_weight)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index, edge_weight)
        z = self.encoder(x, edge_index, edge_weight)
        z1 = self.encoder(x1, edge_index1, edge_weight1)
        z2 = self.encoder(x2, edge_index2, edge_weight2)
        return z, z1, z2

    def project(self, z: torch.Tensor) -> torch.Tensor:
        z = F.elu(self.fc1(z))
        return self.fc2(z)


def train(encoder_model, contrast_model, data, optimizer):
    encoder_model.train()
    optimizer.zero_grad()
    z, z1, z2 = encoder_model(data.x, data.edge_index, data.edge_weight)
    h1, h2 = [encoder_model.project(x) for x in [z1, z2]]
    loss = contrast_model(h1, h2)
    loss.backward()
    optimizer.step()
    return loss.item()


def test(encoder_model, data):
    encoder_model.eval()
    z, _, _ = encoder_model(data.x, data.edge_index, data.edge_weight)
    split = get_split(num_samples=z.size()[0], train_ratio=0.6, test_ratio=0.01)
    result = LREvaluator()(z, data.y, split)
    return result


def main():
    device = torch.device('cpu')
    path = osp.join(osp.expanduser('~'), 'datasets')
    #dataset = Planetoid(path, name='Cora', transform=T.NormalizeFeatures())
    data = train_data_list[1].to(device)

    aug1 = A.Compose([A.EdgeRemoving(pe=0.4), A.FeatureMasking(pf=0.4)])
    aug2 = A.Compose([A.EdgeRemoving(pe=0.4), A.FeatureMasking(pf=0.4)])

    gconv = GConv(input_dim=200, hidden_dim=100, activation=torch.nn.ReLU, num_layers=3).to(device)
    encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2), hidden_dim=200, proj_dim=100).to(device)
    contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L', intraview_negs=True).to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.001)

    with tqdm(total=num_epochs, desc='(T)') as pbar:
        for epoch in range(1, num_epochs+1):
            loss = train(encoder_model, contrast_model, data, optimizer)
            pbar.set_postfix({'loss': loss})
            pbar.update()

    test_result = test(encoder_model, data)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')
    tr = test_result
    print(test_result)
    
    return encoder_model

if __name__ == '__main__':
    #e_m=main() 
    '''device = torch.device('cpu')
    aug1 = A.Compose([A.EdgeRemoving(pe=0.3), A.FeatureMasking(pf=0.3)])
    aug2 = A.Compose([A.EdgeRemoving(pe=0.3), A.FeatureMasking(pf=0.3)])

    gconv = GConv(input_dim=200, hidden_dim=200, activation=torch.nn.ReLU, num_layers=3).to(device)
    e_m = Encoder(encoder=gconv, augmentor=(aug1, aug2), hidden_dim=200, proj_dim=100).to(device)'''

In [ ]:
data_tr_list, data_trte_list, trte_idx, labels_trte = prepare_trte_data(data_folder, view_list)
adj_parameter = 2
adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)

In [ ]:
data_te_list = []
for i in range(len(data_trte_list)):
    cur_d = data_trte_list[i]
    cur_x = cur_d[trte_idx['te']]
    data_te_list.append(cur_x)

In [ ]:
class Graph_Data(Dataset):
    def __init__(self,D,adj_o):
        super().__init__()
        self.D = D
        self.adj_o = adj_o
        
    def __len__(self):
        return 1
    
    def __getitem__(self,idx):
        return self.D,self.adj_o
        
        
        

In [ ]:
td = Graph_Data(data_tr_list[1],adj_tr_list[1])
vd = Graph_Data(data_trte_list[1],adj_te_list[1])

In [ ]:
def collate_batch(batch):
    return batch

In [ ]:
tl = DataLoader(td,batch_size=1,collate_fn = collate_batch)
vl = DataLoader(vd,batch_size=1,collate_fn = collate_batch)

In [ ]:
accuracy = Accuracy()

In [ ]:
class LitModel(pl.LightningModule):
    def __init__(self,lr,e_m):
        super().__init__()
        self.layers = [torch.nn.Linear(200,100),
                       torch.nn.LeakyReLU(),
                       torch.nn.Dropout(p=0.3),
                       torch.nn.Linear(100,50),
                       torch.nn.LeakyReLU(),
                       torch.nn.Dropout(p=0.3),
                       torch.nn.Linear(50,2)]
        self.model = torch.nn.Sequential(*self.layers)
        self.lr = lr
        self.e_m = e_m

    def forward(self, x,edge_index,edge_weight):
        return self.e_m(x,edge_index,edge_weight)

    def training_step(self, batch, batch_idx):
        d_obj, adj_obj = batch[0]
        z,_,_ = self(d_obj,adj_obj.coalesce().indices(),adj_obj.coalesce().values())
        nn_logits = self.model(z)
        y = torch.tensor(labels_trte[trte_idx['tr']]).long()
        loss = F.cross_entropy(nn_logits, y)
        acc = accuracy(nn_logits,y)
        self.log("Training_Loss",loss)
        self.log("Training_Accuracy",acc)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        d_obj, adj_obj = batch[0]
        z,_,_ = self(d_obj,adj_obj.coalesce().indices(),adj_obj.coalesce().values())
        nn_logits = self.model(z)[trte_idx['te']]
        y = torch.tensor(labels_trte[trte_idx['te']]).long()
        loss = F.cross_entropy(nn_logits, y)
        acc = accuracy(nn_logits,y)
        self.log("Validation_Loss",loss)
        self.log("Validation_Accuracy",acc)
        
        
        return loss
    
    
    def test_step(self, batch, batch_idx):
        d_obj, adj_obj = batch[0]
        z,_,_ = self(d_obj,adj_obj.coalesce().indices(),adj_obj.coalesce().values())
        nn_logits = self.model(z)[trte_idx['te']]
        y = torch.tensor(labels_trte[trte_idx['te']]).long()
        loss = F.cross_entropy(nn_logits, y)
        acc = accuracy(nn_logits,y)
        test_acc.append(acc)
        self.log("Test_Loss",loss)
        self.log("Test_Accuracy",acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [ ]:
#sup_model = LitModel(0.001,e_m)

In [ ]:
'''trainer = Trainer(accelerator = 'cpu',
                  logger=wandb_logger,
                  max_epochs=100,
                  log_every_n_steps=1)'''

In [ ]:
'''trainer.fit(sup_model,tl,vl)
trainer.test(sup_model,vl)
wandb.finish()'''

____________________________________________________________

In [ ]:
num_epochs = 2000
class GConv(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, activation, num_layers):
        super(GConv, self).__init__()
        self.activation = activation()
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(input_dim, hidden_dim, cached=False))
        for _ in range(num_layers - 1):
            self.layers.append(GCNConv(hidden_dim, hidden_dim, cached=False))

    def forward(self, x, edge_index, edge_weight=None):
        z = x
        for i, conv in enumerate(self.layers):
            z = conv(z, edge_index, edge_weight)
            z = self.activation(z)
            if i != len(self.layers)-1:
                z = F.dropout(z,p=0.1)
        return z


class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor, hidden_dim, proj_dim):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

        self.fc1 = torch.nn.Linear(hidden_dim, proj_dim)
        self.fc2 = torch.nn.Linear(proj_dim, hidden_dim)

    def forward(self, x, edge_index, edge_weight=None):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index, edge_weight)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index, edge_weight)
        z = self.encoder(x, edge_index, edge_weight)
        z1 = self.encoder(x1, edge_index1, edge_weight1)
        z2 = self.encoder(x2, edge_index2, edge_weight2)
        return z, z1, z2

    def project(self, z: torch.Tensor) -> torch.Tensor:
        z = F.elu(self.fc1(z))
        return self.fc2(z)


def train(encoder_model, contrast_model, data, optimizer):
    encoder_model.train()
    optimizer.zero_grad()
    z, z1, z2 = encoder_model(data.x, data.edge_index, data.edge_weight)
    h1, h2 = [encoder_model.project(x) for x in [z1, z2]]
    loss = contrast_model(h1, h2)
    loss.backward()
    optimizer.step()
    return loss.item()


def test(encoder_model, data):
    encoder_model.eval()
    z, _, _ = encoder_model(data.x, data.edge_index, data.edge_weight)
    split = get_split(num_samples=z.size()[0], train_ratio=0.6, test_ratio=0.01)
    result = LREvaluator()(z, data.y, split)
    return result


def main(model_no):
    
    device = torch.device('cpu')
    path = osp.join(osp.expanduser('~'), 'datasets')
    #dataset = Planetoid(path, name='Cora', transform=T.NormalizeFeatures())
    data = train_data_list[model_no].to(device)

    aug1 = A.Compose([A.EdgeRemoving(pe=0.4), A.FeatureMasking(pf=0.4)])
    aug2 = A.Compose([A.EdgeRemoving(pe=0.4), A.FeatureMasking(pf=0.4)])
    
    if model_no == 2:
        gconv = GConv(input_dim=503, hidden_dim=100, activation=torch.nn.ReLU, num_layers=1).to(device)
        encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2), hidden_dim=100, proj_dim=100).to(device)
        contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L', intraview_negs=True).to(device)
        
    else:
        gconv = GConv(input_dim=1000, hidden_dim=100, activation=torch.nn.ReLU, num_layers=1).to(device)
        encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2), hidden_dim=100, proj_dim=100).to(device)
        contrast_model = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L', intraview_negs=True).to(device)

    optimizer = Adam(encoder_model.parameters(), lr=0.0001)

    with tqdm(total=num_epochs, desc='(T)') as pbar:
        for epoch in range(1, num_epochs+1):
            loss = train(encoder_model, contrast_model, data, optimizer)
            pbar.set_postfix({'loss': loss})
            pbar.update()

    '''test_result = test(encoder_model, data)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')
    tr = test_result
    print(test_result)'''
    
    return encoder_model

if __name__ == '__main__':
    e_m_MRNA=main(0) 
    e_m_meth=main(1)
    e_m_miRNA=main(2)
    '''device = torch.device('cpu')
    aug1 = A.Compose([A.EdgeRemoving(pe=0.3), A.FeatureMasking(pf=0.3)])
    aug2 = A.Compose([A.EdgeRemoving(pe=0.3), A.FeatureMasking(pf=0.3)])

    gconv = GConv(input_dim=200, hidden_dim=200, activation=torch.nn.ReLU, num_layers=3).to(device)
    e_m = Encoder(encoder=gconv, augmentor=(aug1, aug2), hidden_dim=200, proj_dim=100).to(device)'''

In [ ]:
accuracy = Accuracy()
f1 = F1Score(num_classes = 5,average = 'weighted')
auroc = F1Score(num_classes=5,average = 'macro')

In [ ]:
class LitModel(pl.LightningModule):
    ''' torch.nn.Linear(100,50),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(p=0.3),'''
    def __init__(self,lr,e_m_0,e_m_1,e_m_2):
        super().__init__()
        self.layers = [torch.nn.Linear(300,100),
                       torch.nn.LeakyReLU(),
                       torch.nn.Dropout(p=0.1),
                       torch.nn.Linear(100,5)]
        self.model = torch.nn.Sequential(*self.layers)
        self.lr = lr
        self.e_m_mRNA = e_m_0
        self.e_m_meth = e_m_1
        self.e_m_miRNA = e_m_2

    def forward(self, x,model_no,edge_index,edge_weight):
        if model_no == 0:
            return self.e_m_mRNA(x,edge_index,edge_weight)
        elif model_no == 1:
            return self.e_m_meth(x,edge_index,edge_weight)
        elif model_no == 2:
            return self.e_m_miRNA(x,edge_index,edge_weight)
        else:
            return "Wrong Model"

    def training_step(self, batch, batch_idx):
        d_obj_0, adj_obj_0,d_obj_1, adj_obj_1,d_obj_2, adj_obj_2 = batch[0]
        z_0,_,_ = self(d_obj_0,0,adj_obj_0.coalesce().indices(),adj_obj_0.coalesce().values())
        z_1,_,_ = self(d_obj_1,1,adj_obj_1.coalesce().indices(),adj_obj_1.coalesce().values())
        z_2,_,_ = self(d_obj_2,2,adj_obj_2.coalesce().indices(),adj_obj_2.coalesce().values())
        z = torch.cat([z_0,z_1,z_2],axis=1)
        nn_logits = self.model(z)
        y = torch.tensor(labels_trte[trte_idx['tr']]).long()
        loss = F.cross_entropy(nn_logits, y)
        acc = accuracy(nn_logits,y)
        f1_s = f1(nn_logits,y)
        auc_s = auroc(nn_logits,y)
        self.log("Training_Loss",loss)
        self.log("Training_Accuracy",acc)
        self.log("Training_F1_weighted",f1_s)
        self.log("Training_F1_macro",auc_s)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        d_obj_0, adj_obj_0,d_obj_1, adj_obj_1,d_obj_2, adj_obj_2 = batch[0]
        z_0,_,_ = self(d_obj_0,0,adj_obj_0.coalesce().indices(),adj_obj_0.coalesce().values())
        z_1,_,_ = self(d_obj_1,1,adj_obj_1.coalesce().indices(),adj_obj_1.coalesce().values())
        z_2,_,_ = self(d_obj_2,2,adj_obj_2.coalesce().indices(),adj_obj_2.coalesce().values())
        z = torch.cat([z_0,z_1,z_2],axis=1)
        nn_logits = self.model(z)[trte_idx['te']]
        y = torch.tensor(labels_trte[trte_idx['te']]).long()
        loss = F.cross_entropy(nn_logits, y)
        acc = accuracy(nn_logits,y)
        f1_s = f1(nn_logits,y)
        auc_s = auroc(nn_logits,y)
        self.log("Validation_Loss",loss)
        self.log("Validation_Accuracy",acc)
        self.log("Validation_F1_weighted",f1_s)
        self.log("Validation_F1_macro",auc_s)
        
        
        return loss
    
    
    def test_step(self, batch, batch_idx):
        d_obj_0, adj_obj_0,d_obj_1, adj_obj_1,d_obj_2, adj_obj_2 = batch[0]
        z_0,_,_ = self(d_obj_0,0,adj_obj_0.coalesce().indices(),adj_obj_0.coalesce().values())
        z_1,_,_ = self(d_obj_1,1,adj_obj_1.coalesce().indices(),adj_obj_1.coalesce().values())
        z_2,_,_ = self(d_obj_2,2,adj_obj_2.coalesce().indices(),adj_obj_2.coalesce().values())
        z = torch.cat([z_0,z_1,z_2],axis=1)
        nn_logits = self.model(z)[trte_idx['te']]
        y = torch.tensor(labels_trte[trte_idx['te']]).long()
        loss = F.cross_entropy(nn_logits, y)
        acc = accuracy(nn_logits,y)
        f1_s = f1(nn_logits,y)
        auc_s = auroc(nn_logits,y)
        self.log("Test_Loss",loss)
        self.log("Test_Accuracy",acc)
        self.log("Test_F1_weighted",f1_s)
        self.log("Test_F1_macro",auc_s)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [ ]:
class Graph_Data(Dataset):
    def __init__(self,D,adj_o):
        super().__init__()
        self.D = D
        self.adj_o = adj_o
        
    def __len__(self):
        return 1
    
    def __getitem__(self,idx):
        return self.D[0],self.adj_o[0],self.D[1],self.adj_o[1],self.D[2],self.adj_o[2]
        
        
        

In [27]:
data_tr_list, data_trte_list, trte_idx, labels_trte = prepare_trte_data(data_folder, view_list)
adj_parameter = 2
adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)

In [20]:
adj_tr_list[0]

tensor(indices=tensor([[  0,   1,   2,  ..., 609, 610, 611],
                       [  0,   1,   2,  ..., 609, 610, 611]]),
       values=tensor([1.0000, 1.0000, 0.2505,  ..., 1.0000, 1.0000, 1.0000]),
       size=(612, 612), nnz=1226, layout=torch.sparse_coo)

In [ ]:
td = Graph_Data(data_tr_list,adj_tr_list)
vd = Graph_Data(data_trte_list,adj_te_list)
def collate_batch(batch):
    return batch

tl = DataLoader(td,batch_size=1,collate_fn = collate_batch)
vl = DataLoader(vd,batch_size=1,collate_fn = collate_batch)


In [ ]:
comp_model = LitModel(0.00001,e_m_MRNA,e_m_meth,e_m_miRNA)

In [ ]:
trainer_1 = Trainer(accelerator = 'cpu',
                  logger=wandb_logger,
                  max_epochs=7111,
                  log_every_n_steps=1)

In [ ]:
trainer_1.fit(comp_model,tl,vl)
trainer_1.test(comp_model,vl)
wandb.finish()